In [30]:
import pandas as pd
import numpy as np
import scipy
import lifelines
import pandas
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

# Load datasets
merged_data = pd.read_csv("path/to/cohort_file.csv")
merged_data_key = pd.read_csv("path/to/cohort_file.csv")

# Create and apply a column renaming mapping for merged_data
cohort_mapping = dict(zip(merged_data_key["Column Name"], merged_data_key["Description"]))
cohort_mapping.pop('PID', None)  # Ensure 'PID' remains unchanged
trimmed_cohort_mapping = {col: desc.split(" - ")[-1] for col, desc in cohort_mapping.items()}
merged_data.rename(columns=trimmed_cohort_mapping, inplace=True)

# Calculate overlap weights
merged_data['overlap_weight'] = merged_data['Propensity Score']
merged_data.loc[merged_data['Exposure Group'] == 'E', 'overlap_weight'] = 1 - merged_data['Propensity Score']

# Inspect the final dataset
merged_data

In [ ]:
# Compute Effective Sample Size (ESS_optum)
sum_weights = merged_data["overlap_weight"].sum()
sum_weights_squared = (merged_data["overlap_weight"] ** 2).sum()
ESS_optum = (sum_weights ** 2) / sum_weights_squared

print(f"Computed Effective Sample Size (ESS_optum): {ESS_optum:.2f}")

# Normalize weights to ensure sum equals ESS_optum
scaling_factor = ESS_optum / sum_weights
merged_data["overlap_weight"] = merged_data["overlap_weight"] * scaling_factor

In [ ]:
import numpy as np
import pandas as pd

# Individual databases
datasets = {
    'Optum': merged_data,
}

# Overall weighted mean and SD per database
for name, data in datasets.items():
    mean = np.average(data['Follow Up Time'], weights=data['overlap_weight'])
    variance = np.average((data['Follow Up Time'] - mean) ** 2, weights=data['overlap_weight'])
    sd = np.sqrt(variance)
    
    print(f"{name} Weighted Mean Follow-up Time: {mean:.1f}")
    print(f"{name} Weighted SD of Follow-up Time: {sd:.1f}\n")
    
    # Stratified by Exposure Group
    print(f"{name} Stratified Weighted Follow-up Time by Exposure Group:")
    for group, group_df in data.groupby("Exposure Group"):
        group_mean = np.average(group_df["Follow Up Time"], weights=group_df["overlap_weight"])
        group_variance = np.average((group_df["Follow Up Time"] - group_mean) ** 2, weights=group_df["overlap_weight"])
        group_sd = np.sqrt(group_variance)

        print(f"  Group {group}: Mean = {group_mean:.1f} days, SD = {group_sd:.1f} days")
    print()